In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
# https://mokeya.tistory.com/m/68

In [4]:
# 함수 정의: 검색어 조건에 따른 url 생성
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

In [5]:
insta_searching('여행')

'https://www.instagram.com/explore/tags/여행'

In [6]:
# 검색 결과 페이지에서 첫 번째 게시물을 클릭하는 함수를 정의.

# 인스타그램 서버에서 우리 머신이 (무언가 불순한 의도를 가지고) 마우스를 갈기는 것처럼 인식되지 않도록,

# 또 데이터를 긁어오는 시간을 충분히 확보할 수 있도록,

# 다음 행동과의 시차를 time.sleep() 메서드로 구현

In [8]:
# 함수 정의: 열린 페이지에서 첫 번째 게시물 클릭 + sleep 메소드 통하여 시차 두기
def select_first(driver):
    first = driver.find_elements_by_css_selector("div._9AhH0")[0]
    first.click()
    time.sleep(3)

In [11]:
# 함수 정의: 본문 내용, 작성일자, 좋아요 수, 위치 정보, 해시태그 가져오기
import re
from bs4 import BeautifulSoup

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용
    try:
        content = soup.select('div.MOdxS')[0].text
    except:
        content = ''
    # 해시태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 작성일자
    date = soup.select('time._1o9PC')[0]['datetime'][:10]
    
    # 좋아요
    try:
        like = soup.select('section.EDfFK.ygqzn')[0].findAll('span')[-1].text
    except:
        like = 0
    # 위치
    try:
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''

    data = [content, date, like, place, tags]
    return data


In [12]:
def move_next(driver):
    right = driver.find_element_by_css_selector("div.l8mY4.feth3") # 2022/01/11 수정
    right.click()
    time.sleep(3)

In [20]:
# 크롤링 시작
"""
driver.get(url)을 통해 검색 페이지 접속하고,
target 변수에 크롤링할 게시글의 수를 바인딩
"""

# 크롬 브라우저 열기
driver = webdriver.Edge('C:/Program Files/msedgedriver.exe')

driver.get('https://www.instagram.com')
time.sleep(3)

# 인스타그램 로그인을 위한 계정 정보
email = '인스타그램 로그인 아이디'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '인스타그램 로그인 비번'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

# 게시물을 조회할 검색 키워드 입력 요청
word = input("검색어를 입력하세요 : ")
word = str(word)
url = insta_searching(word)

# 검색 결과 페이지 열기
driver.get(url)
time.sleep(8) # 코드 수행 환경에 따라 페이지가 로드되는 데 시간이 더 걸릴 수 있어 8초로 변경(2022/01/11)

# 첫 번째 게시물 클릭
select_first(driver)

# 본격적으로 데이터 수집 시작
results = []
## 수집할 게시물의 수
target = 10000
for i in range(target):

    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)

print(results[:2])

C:\Users\LG\AppData\Local\Temp\ipykernel_11892\1247328928.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('C:/Program Files/msedgedriver.exe')


AttributeError: 'WebDriver' object has no attribute 'find_elements_by_css_selector'